In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np
import cv2

In [2]:
def scale(img, nR, nC):
    height = img.shape[0]     # source number of rows 
    width = img.shape[1]  # source number of columns 
    return np.array([[img[int(height * r / nR)][int(width * c / nC)]  for c in range(nC)] for r in range(nR)])

In [3]:
def gaussian2d(x,y,sigma):
    """Function for calculating the 2-dimensional gaussian"""
    x2 = x**2
    y2 = y**2
    sigma2 = sigma**2
    # Return the value of 2-dimensional gaussian value with the given point x, y
    return (1/(2*np.pi*sigma2))*np.exp(-1*((x2+y2)/(2*sigma2)))

In [4]:
def get_gaussian_kernel(size, sigma=None):
    """
    Getting the gaussian kernel
    The effcient size for the gaussian kernel is the smallest odd number that is larger than 6*sigma     
    """
    # Calculating the best sigma for the given size
    if sigma == None:
        sigma = 0.3 * (size // 2) + 0.8
    # Getting the range of x, y
    aran = np.arange(-1 * (size // 2), size // 2 + 1)
    XX, YY = np.meshgrid(aran, aran)
    # making the kernel
    kernel = gaussian2d(XX, YY, sigma)
    #normaliing
    kernel = kernel/np.sum(kernel)
    return kernel

In [5]:
def filter(weights,roi):
    weights = weights.astype('float')
    roi = roi.astype('float')
    # this holds the end result
    filtered = np.zeros_like(roi)
    width = int((weights.shape[1]-1)/2)
    height = int((weights.shape[0]-1)/2)

    # convolution
    for i in range(height,roi.shape[1]-height):
        for j in range(width,roi.shape[0]-width):
            filtered[j,i]= np.sum(weights * roi[j-width:j+width+1, i-height:i+height+1])
    
    return filtered

In [17]:
def getFeaturePoints(img, winSize=7, threshold=0.25):
    h, w = img.shape[:2]
    # Doing the derivatives
    filter_sobelx = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
    ])
    I_x=filter(filter_sobelx, img)

    filter_sobely = np.array([
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1]
    ])
    I_y=filter(filter_sobely, img)

    g_kernel = get_gaussian_kernel(winSize)

    # Making the matrix H for each pixel
    H = np.zeros((h,w,2,2))
    H[:, :, 0, 0] = filter(g_kernel, np.square(I_x))
    H[:, :, 0, 1] = filter(g_kernel, I_x*I_y)
    H[:, :, 1, 0] = filter(g_kernel, I_x*I_y)
    H[:, :, 1, 1] = filter(g_kernel, np.square(I_y))

    # Making a matrix for holding the values for corner_strength
    corner_strength = np.zeros((h,w))

    coords = []

    # usually the value 0.04 is the optimal value
    k = 0.04
    for idx_h in range(h):
        for idx_w in range(w):
            det = np.linalg.det(H[idx_h, idx_w, :, :])
            tr = np.trace(H[idx_h, idx_w, :, :])
            harris = det - (k*(tr**2))
            corner_strength[idx_h][idx_w] = harris

            if harris >= threshold:
                coords.append((idx_h, idx_w))

    coords = list(set(coords))

    print('='*20,"Plotting",'='*20)
    
    fig, ax = plt.subplots()

    ax.imshow(img,cmap="gray")
    
    # Plotting the circle
    for blob in coords:
        y, x = blob
        c = plt.Circle((x, y), 2, linewidth=1.5 ,color='red', fill=False)
        ax.add_patch(c)
    ax.plot()  
    plt.show()


    return coords

In [7]:
def loadImages(path):
    images = []
    for i in range(1,19):
        img = plt.imread(path+'/image'+str(i)+'.png')
        img = img.astype('float')
        # Converting the image to gray scale
        R, G, B = img[:,:,0], img[:,:,1], img[:,:,2]
        img = 0.2989 * R + 0.5870 * G + 0.1140 * B
        img = scale(img, 800, 1000)
        images.append(img)

    return images

In [8]:
images = loadImages('./images')